# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [3]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,iqaluit,63.7506,-68.5145,7.85,75,75,3.09,CA,1723605775
1,1,alice springs,-23.7000,133.8833,24.75,18,0,3.09,AU,1723606016
2,2,waitangi,-43.9535,-176.5597,8.79,92,98,2.68,NZ,1723606017
3,3,st. john's,47.5649,-52.7093,12.76,95,100,6.71,CA,1723606019
4,4,adamstown,-25.0660,-130.1015,21.89,86,100,7.45,PN,1723606020


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng', 'Lat', 
    geo=True, 
    tiles='OSM', 
    color='City',  # Color by city name or another category if you prefer
    size='Humidity',  # Size of the point corresponds to humidity
    hover_cols=['City', 'Country', 'Humidity']  # Columns to display when hovering over a point
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df.loc[
    (city_data_df["Max Temp"] < 70) & 
    (city_data_df["Max Temp"] > 20) &  
    (city_data_df['Wind Speed'] < 10) & 
    (city_data_df['Cloudiness'] < 20)
]
# Drop any rows with null values
ideal_weather_df.dropna()

# Display sample data
ideal_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,alice springs,-23.7000,133.8833,24.75,18,0,3.09,AU,1723606016
15,15,cannonvale,-20.2833,148.7000,26.99,55,0,3.09,AU,1723606037
18,18,rouissat,31.9243,5.3502,30.04,20,7,1.03,DZ,1723606043
28,28,antsiranana,-12.2787,49.2917,21.69,79,15,7.78,MG,1723606060
30,30,dixon,38.4455,-121.8233,27.32,45,0,4.12,US,1723605862


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
1,alice springs,AU,-23.7000,133.8833,18,
15,cannonvale,AU,-20.2833,148.7000,55,
18,rouissat,DZ,31.9243,5.3502,20,
28,antsiranana,MG,-12.2787,49.2917,79,
30,dixon,US,38.4455,-121.8233,45,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Set parameters to search for a hotel
radius = 10000  # Radius in meters
api_key = geoapify_key  # Assuming you have your Geoapify API key stored in ⁠ geoapify_key ⁠
params = {
    'categories': 'accommodation.hotel',  # Searching for hotels
    'limit': 1,  # Limit to the first hotel found
    'apiKey': api_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lng},{lat},{radius}'
    params["bias"] = f'proximity:{lng},{lat}'

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary --- name_address = # YOUR CODE HERE checar
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
alice springs - nearest hotel: Aurora Alice Springs
cannonvale - nearest hotel: Whitsundays Rainforest Retreat
rouissat - nearest hotel: فندق الأروي الذهبي
antsiranana - nearest hotel: Central
dixon - nearest hotel: Best Western Plus Dixon Davis
albany - nearest hotel: No hotel found
jackson - nearest hotel: Brookside
mount isa - nearest hotel: Ibis Styles
guerrero negro - nearest hotel: Plaza sal paraiso
gaalkacyo - nearest hotel: Citycenter وسط المدينة
utrik - nearest hotel: No hotel found
morehead - nearest hotel: No hotel found
mantua - nearest hotel: No hotel found
san jose - nearest hotel: Hotel De Anza
mhamid - nearest hotel: Hotel Kasbah Azalay
lethem - nearest hotel: Hotel Amazonas
xilin hot - nearest hotel: Hotel hailibu 120
mazar-e sharif - nearest hotel: هوتل برات
kalianget - nearest hotel: No hotel found
el haouaria - nearest hotel: Villa Zembra Maison d'hôtes
fale old settlement - nearest hotel: No hotel found
buka - nearest hotel: No hotel found
aca

,City,Country,Lat,Lng,Humidity,Hotel Name
1,alice springs,AU,-23.7000,133.8833,18,Aurora Alice Springs
15,cannonvale,AU,-20.2833,148.7000,55,Whitsundays Rainforest Retreat
18,rouissat,DZ,31.9243,5.3502,20,فندق الأروي الذهبي
28,antsiranana,MG,-12.2787,49.2917,79,Central
30,dixon,US,38.4455,-121.8233,45,Best Western Plus Dixon Davis


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    'Lng', 'Lat', 
    geo=True, 
    tiles='OSM', 
    color='City',  # Color by city name or another category if you prefer
    size='Humidity',  # Size of the point corresponds to humidity
    hover_cols=['City', 'Country', 'Humidity', 'Hotel Name']  # Include Hotel Name and Country in the hover message
)
# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)